In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
def getHtml(search):
    browser = webdriver.Chrome()
    browser.get('https://apps.webofknowledge.com/')
    time.sleep(1)
    browser.find_element_by_name('value(input1)').send_keys(search)
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="searchCell1"]/span[1]/button').click()
    return browser.page_source

In [11]:
def getUrls(html):
    soup=BeautifulSoup(html,'html.parser')
    urls=[]
    pattern= soup.find('a', attrs={'class': 'smallV110 snowplow-full-record'}).get('href').strip()
    for i in range(1,3):
        for j in range(i*10-9,i*10+1):
            urls.append("https://apps.webofknowledge.com//"+pattern.replace('&page=1&doc=1','&page={}&doc={}'.format(i,j)))
    return urls

In [12]:
def getImformation(urls):
    list=[]
    headers={
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'
        }
    for url in urls:
        data={}
        response=requests.get(url,headers=headers).text
        soup=BeautifulSoup(response)
        data['文献名称']=soup.find('div',attrs={'class':'title'}).text.strip()
        
        authors=[]
        for author in soup.find_all('a',attrs={'title':'Find more records by this author'}):
            authors.append(author.text.strip())
        data['作者']=authors

        data['来源']= soup.find(attrs={'class':'sourceTitle'}).text.strip()
        
        
        keywords=[]
        if soup.find('a',attrs={'title':'Find more records by this author keywords'}):
            for keyword in soup.find_all('a',attrs={'title':'Find more records by this author keywords'}):
                keywords.append(keyword.text.strip())
        else:
            keywords=str(soup.select('div.l-content div:nth-of-type(5) p')[0].text).replace('Keyword List:','').replace('\n','')
        data['关键字']=keywords
        
        list.append(data)
    df=pd.DataFrame(list)
    print(df)
#     writer = pd.ExcelWriter('E:/3.xlsx')
#     df.to_excel(writer,'Sheet1')
#     writer.save()

In [13]:
def main():
    search=input('请输入需要查找的关键字：')
    getImformation(getUrls(getHtml(search)))
    
main()

请输入需要查找的关键字：5G
                                                   作者  \
0               [Yeganeh, H, Salahi, A, Pourmina, MA]   
1   [Varela-Martinez, DA, Gonzalez-Curbelo, MA, Go...   
2   [Saber, MG, Vall-Llosera, G, Patel, D, Samani,...   
3   [Booth, M, Broderick, N, Chen, L, Kim, M, Peac...   
4   [Shen, C, Holguin-Lerma, JA, Alatawi, AA, Zou,...   
5               [Liu, H, Zhao, L, Diao, HW, Wang, WJ]   
6                  [Buene, AF, Hagfeldt, A, Hoff, BH]   
7   [Zhao, T, Zhou, Y, Li, WY, Ma, XY, Zhan, RT, C...   
8   [Diby, ND, Duan, YQ, Grah, PA, Cai, FS, Yuan, ZH]   
9   [Hernandez-Crespo, C, Fernandez-Gonzalvo, M, M...   
10  [Ozturk, M, Gogate, M, Onireti, O, Adeel, A, H...   
11                           [He, L, Wu, D, Tong, MP]   
12  [Verma, VK, Patel, RK, Deshmukh, NA, Jha, AK, ...   
13  [Deng, RX, Chen, BB, Li, HG, Zhang, K, Zhang, ...   
14  [Kubo-Kaneda, M, Tanaka, H, Maki, S, Nii, M, U...   
15                  [Chawla, P, Sharma, SK, Toor, AP]   
16              

In [ ]:
url='http://apps.webofknowledge.com/full_record.do?product=UA&search_mode=GeneralSearch&qid=1&SID=5AqXdK9zdUxD5a6hmWg&page=1&doc=1'
response=requests.get(url).text
so=BeautifulSoup(response,'html.parser')

In [ ]:
print(so.select('div.l-content  div:nth-of-type(5) p')[0].text)